### Loading Libraries & Environment Variables

In [2]:
import os
import requests
import json
import prettytable
import pandas as pd
from dotenv import load_dotenv

In [3]:
# Load the environment variables from the .env file
#by calling the load_dotenv function
load_dotenv()

True

### Loading Data

#### Setting up to use the BLS API

In [4]:
# Set the base url for BLS API v2
url = 'https://api.bls.gov/publicAPI/v2/timeseries/data/'

In [5]:
# Pull the API key in .env which contains: bls_key = 'key'
bls_key = os.getenv("bls_key")
key = '?registrationkey={}'.format(bls_key)

In [6]:
# Set the start year and end year for the 5-year data series
dates = ('2019', '2021')

#### Loading Labor Force data for Washington Counties

In [11]:
# Creating series list
# example: 'LAUCN530010000000005'
# The series id has the following structure:
# 'LA' = Prefix for local area data
# 'U' (or 'S') for seasonal adjustment 
# 'CN53001000000' = area code --> 'CN' for county level data + state fips code + county fips code
# '06' = 2-digit measure code for Labor Force

# Create a dictionary of the series codes and fips codes
series_dict = {
    'LAUCN530010000000006': '53001',
    'LAUCN530030000000006': '53003',
    'LAUCN530050000000006': '53005',
    'LAUCN530070000000006': '53007',
    'LAUCN530090000000006': '53009',
    'LAUCN530110000000006': '53011',
    'LAUCN530130000000006': '53013',
    'LAUCN530150000000006': '53015',
    'LAUCN530170000000006': '53017',
    'LAUCN530190000000006': '53019',
    'LAUCN530210000000006': '53021',
    'LAUCN530230000000006': '53023',
    'LAUCN530250000000006': '53025',
    'LAUCN530270000000006': '53027',
    'LAUCN530290000000006': '53029',
    'LAUCN530310000000006': '53031',
    'LAUCN530330000000006': '53033',
    'LAUCN530350000000006': '53035',
    'LAUCN530370000000006': '53037',
    'LAUCN530390000000006': '53039',
    'LAUCN530410000000006': '53041',
    'LAUCN530430000000006': '53043',
    'LAUCN530450000000006': '53045',
    'LAUCN530470000000006': '53047',
    'LAUCN530490000000006': '53049',
    'LAUCN530510000000006': '53051',
    'LAUCN530530000000006': '53053',
    'LAUCN530550000000006': '53055',
    'LAUCN530570000000006': '53057',
    'LAUCN530590000000006': '53059',
    'LAUCN530610000000006': '53061',
    'LAUCN530630000000006': '53063',
    'LAUCN530650000000006': '53065',
    'LAUCN530670000000006': '53067',
    'LAUCN530690000000006': '53069',
    'LAUCN530710000000006': '53071',
    'LAUCN530730000000006': '53073',
    'LAUCN530750000000006': '53075',
    'LAUCN530770000000006': '53077'
}

# The following code is modeled from an example provided at https://www.bd-econ.com/blsapi.html.

# Specify json as content type to return
headers = {'Content-type': 'application/json'}

# Submit the list of series and the dates as data
data = json.dumps({
    "seriesid": list(series_dict.keys()),
    "startyear": dates[0],
    "endyear": dates[1]})


In [19]:
#The API is currently retricting the call to 25 series at a time. Use this dictionary to pull a second file with the remaining 14 wa counties.
series_dict = {
    'LAUCN530510000000006': '53051',
    'LAUCN530530000000006': '53053',
    'LAUCN530550000000006': '53055',
    'LAUCN530570000000006': '53057',
    'LAUCN530590000000006': '53059',
    'LAUCN530610000000006': '53061',
    'LAUCN530630000000006': '53063',
    'LAUCN530650000000006': '53065',
    'LAUCN530670000000006': '53067',
    'LAUCN530690000000006': '53069',
    'LAUCN530710000000006': '53071',
    'LAUCN530730000000006': '53073',
    'LAUCN530750000000006': '53075',
    'LAUCN530770000000006': '53077'
}

# Specify json as content type to return
headers = {'Content-type': 'application/json'}

# Submit the list of series and the dates as data
data = json.dumps({
    "seriesid": list(series_dict.keys()),
    "startyear": dates[0],
    "endyear": dates[1]})

In [20]:
# Posting request for the data
p = requests.post(
    '{}{}'.format(url, key),
    headers=headers,
    data=data).json()['Results']['series']

In [21]:
# Creating a date index from first series
date_list = [f"{i['year']}-{i['period'][1:]}-01" for i in p[0]['data']]

# Creating an empty dataframe to fill with values
df_labor_force = pd.DataFrame()

# Building a pandas series from the API results, p
for s in p:
    df_labor_force[series_dict[s['seriesID']]] = pd.Series(
        index = pd.to_datetime(date_list),
        data = [i['value'] for i in s['data']]
        ).astype(float).iloc[::-1]


In [22]:
#Reviewing labor force dataframe
df_labor_force.head()

,53051,53053,53055,53057,53059,53061,53063,53065,53067,53069,53071,53073,53075,53077
2019-01-01,5006.0,437184.0,7858.0,62053.0,5602.0,436333.0,252794.0,18916.0,141009.0,1364.0,29006.0,114376.0,24337.0,125062.0
2019-02-01,5069.0,440179.0,7855.0,62689.0,5664.0,440097.0,255176.0,19091.0,142323.0,1379.0,29275.0,115417.0,24617.0,125033.0
2019-03-01,5042.0,440261.0,7935.0,62704.0,5604.0,439304.0,254947.0,19141.0,142478.0,1385.0,29655.0,115534.0,24630.0,127033.0
2019-04-01,4971.0,435853.0,8131.0,62558.0,5528.0,435333.0,252117.0,18895.0,141312.0,1346.0,29747.0,114713.0,24587.0,128182.0
2019-05-01,4938.0,435080.0,8472.0,62607.0,5513.0,437135.0,252188.0,18704.0,140670.0,1362.0,29636.0,114215.0,24454.0,129458.0


In [23]:
#Saving the first file to a csv
df_labor_force.to_csv('Resources/wa_labor_force_by_county1.csv')

In [ ]:
#Saving the second file to a csv
df_labor_force.to_csv('Resources/wa_labor_force_by_county2.csv')

### Loading Employment data for Washington Counties

In [24]:
# Creating series list
# example: 'LAUCN530010000000005'
# The series id has the following structure:
# 'LA' = Prefix for local area data
# 'U' (or 'S') for seasonal adjustment 
# 'CN53001000000' = area code --> 'CN' for county level data + state fips code + county fips code
# '05' = 2-digit measure code for Employment

# Creating a dictionary of the series codes and fips codes
series_dict = {
    'LAUCN530010000000005': '53001',
    'LAUCN530030000000005': '53003',
    'LAUCN530050000000005': '53005',
    'LAUCN530070000000005': '53007',
    'LAUCN530090000000005': '53009',
    'LAUCN530110000000005': '53011',
    'LAUCN530130000000005': '53013',
    'LAUCN530150000000005': '53015',
    'LAUCN530170000000005': '53017',
    'LAUCN530190000000005': '53019',
    'LAUCN530210000000005': '53021',
    'LAUCN530230000000005': '53023',
    'LAUCN530250000000005': '53025',
    'LAUCN530270000000005': '53027',
    'LAUCN530290000000005': '53029',
    'LAUCN530310000000005': '53031',
    'LAUCN530330000000005': '53033',
    'LAUCN530350000000005': '53035',
    'LAUCN530370000000005': '53037',
    'LAUCN530390000000005': '53039',
    'LAUCN530410000000005': '53041',
    'LAUCN530430000000005': '53043',
    'LAUCN530450000000005': '53045',
    'LAUCN530470000000005': '53047',
    'LAUCN530490000000005': '53049',
    'LAUCN530510000000005': '53051',
    'LAUCN530530000000005': '53053',
    'LAUCN530550000000005': '53055',
    'LAUCN530570000000005': '53057',
    'LAUCN530590000000005': '53059',
    'LAUCN530610000000005': '53061',
    'LAUCN530630000000005': '53063',
    'LAUCN530650000000005': '53065',
    'LAUCN530670000000005': '53067',
    'LAUCN530690000000005': '53069',
    'LAUCN530710000000005': '53071',
    'LAUCN530730000000005': '53073',
    'LAUCN530750000000005': '53075',
    'LAUCN530770000000005': '53077'
}

# The following code is modeled from an example provided at https://www.bd-econ.com/blsapi.html.

# Specifying json as content type to return
headers = {'Content-type': 'application/json'}

# Submitting the list of series and the dates as data
data = json.dumps({
    "seriesid": list(series_dict.keys()),
    "startyear": dates[0],
    "endyear": dates[1]})


In [33]:
#This second series dictionary is for getting the last 14 counties becuase the API limits to 25 series in one call.
series_dict = {
    'LAUCN530510000000005': '53051',
    'LAUCN530530000000005': '53053',
    'LAUCN530550000000005': '53055',
    'LAUCN530570000000005': '53057',
    'LAUCN530590000000005': '53059',
    'LAUCN530610000000005': '53061',
    'LAUCN530630000000005': '53063',
    'LAUCN530650000000005': '53065',
    'LAUCN530670000000005': '53067',
    'LAUCN530690000000005': '53069',
    'LAUCN530710000000005': '53071',
    'LAUCN530730000000005': '53073',
    'LAUCN530750000000005': '53075',
    'LAUCN530770000000005': '53077'
}

# Specifying json as content type to return
headers = {'Content-type': 'application/json'}

# Submitting the list of series and the dates as data
data = json.dumps({
    "seriesid": list(series_dict.keys()),
    "startyear": dates[0],
    "endyear": dates[1]})

In [34]:
# Posting request for the data
p = requests.post(
    '{}{}'.format(url, key),
    headers=headers,
    data=data).json()['Results']['series']

In [35]:
# Creating a date index from first series
date_list = [f"{i['year']}-{i['period'][1:]}-01" for i in p[0]['data']]

# Creating an empty dataframe to fill with values for employment
df_employment = pd.DataFrame()

# Building a pandas series from the API results, p
for s in p:
    df_employment[series_dict[s['seriesID']]] = pd.Series(
        index = pd.to_datetime(date_list),
        data = [i['value'] for i in s['data']]
        ).astype(float).iloc[::-1]

In [ ]:
#Saving the first file to csv
df_employment.to_csv('Resources/wa_employment_by_county1.csv')

In [37]:
#Saving the second file to csv
df_employment.to_csv('Resources/wa_employment_by_county2.csv')